#AutoEncoders

##Downloading the dataset

###ML-100K

In [0]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2020-06-08 18:30:02--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.56MB/s    in 1.3s    

2020-06-08 18:30:04 (3.56 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [0]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2020-06-08 18:30:18--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  4.18MB/s    in 1.3s    

2020-06-08 18:30:20 (4.18 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [2]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [3]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


In [4]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [5]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [6]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [17]:
# Stacked Auto Encoders

class StackedAutoEncoders(nn.Module):
    
    def __init__(self,):
        super(StackedAutoEncoders, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
    
        
    

In [18]:
sae = StackedAutoEncoders()

In [19]:
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE


In [21]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    
    for id_user in range(nb_users):
        
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum((target.data > 0) > 0):
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data>0) + 1e-10)
            loss.backward()
            train_loss+= np.sqrt (loss.data * mean_corrector) #error
            s += 1.
            optimizer.step()
    
    print(f"epoch: {epoch} loss: {train_loss/s}")
        
        
        

epoch: 1 loss: 1.764691948890686
epoch: 2 loss: 1.096481204032898
epoch: 3 loss: 1.0534509420394897
epoch: 4 loss: 1.0382879972457886
epoch: 5 loss: 1.0308033227920532
epoch: 6 loss: 1.0268133878707886
epoch: 7 loss: 1.0238819122314453
epoch: 8 loss: 1.021814227104187
epoch: 9 loss: 1.0205382108688354
epoch: 10 loss: 1.0195837020874023
epoch: 11 loss: 1.0186587572097778
epoch: 12 loss: 1.0183454751968384
epoch: 13 loss: 1.017608404159546
epoch: 14 loss: 1.0175540447235107
epoch: 15 loss: 1.0169026851654053
epoch: 16 loss: 1.016904354095459
epoch: 17 loss: 1.0167776346206665
epoch: 18 loss: 1.0164426565170288
epoch: 19 loss: 1.0162432193756104
epoch: 20 loss: 1.0163679122924805
epoch: 21 loss: 1.016167402267456
epoch: 22 loss: 1.0158731937408447
epoch: 23 loss: 1.015805959701538
epoch: 24 loss: 1.015723705291748
epoch: 25 loss: 1.015731930732727
epoch: 26 loss: 1.015626311302185
epoch: 27 loss: 1.0153834819793701
epoch: 28 loss: 1.0150281190872192
epoch: 29 loss: 1.0127474069595337
epoc

## Testing the SAE


In [23]:
test_loss = 0
s = 0.

for id_user in range(nb_users): 
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum((target.data > 0) > 0):
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data>0) + 1e-10)
        test_loss+= np.sqrt (loss.data * mean_corrector) #error
        s += 1.
print(f"test loss: {test_loss/s}")

test loss: 0.9548585414886475
